# Part 3: Other options for input

Umami is a package for calculating metrics for use with for Earth surface dynamics models. This notebook is the final notebook in a three-part introduction to using umami.

Umami was designed to work well with the [terrainbento](https://terrainbento.readthedocs.io/en/latest/) model package, as well as other models built using the [Landlab Toolkit](https://github.com/landlab/landlab). However, umami can be used with models built with other modeling tools and data in a variety of formats. This notebook is meant to demonstrate this capability. 

## Scope of this tutorial

In this tutorial you will learn how to use other input options for umami. 

Specifically we will use square gridded terrain stored as a [netCDF](https://www.unidata.ucar.edu/software/netcdf/).

If you have comments or questions about the notebooks, the best place to get help is through [GitHub Issues](https://github.com/TerrainBento/umami/issues).

To begin this example, we will import the required python packages. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from io import BytesIO, StringIO

import numpy as np
from scipy.interpolate import RegularGridInterpolator
import matplotlib.pylab as plt
from urllib.request import urlopen

import rasterio

from landlab import imshow_grid, RasterModelGrid, VoronoiDelaunayGrid

from umami import Metric

Umami does not make any requirements regarding where terrain data comes from or what model or modeling package is used to construct modeled terrain. However, umami does require that modeled or observed terrain is provided to it as a Landlab grid with an at-node field called `topographic__elevation`. Using the Landlab model grid datastructure means that umami knows how large each grid cell is, and how they are connected. Landlab has five model grid classes. One of them will probably suit your needs. 

- [`RasterModelGrid`]()
- [`HexModelGrid`]()
- [`RadialModelGrid`]()
- [`VoronoiDelaunayGrid`]()
- [`NetworkModelGrid`]() 

In this example we will use the `RasterModelGrid` for regularly spaced square grid cells and the `VoronoiDelaunayGrid` for irregularly spaced observations. 

If you can read your topography into python as a numpy array, you can put it on a Landlab grid called `topographic__elevation`.

You can use a Landlab function such as [`read_esri_ascii`]() or [`read_netcdf`]() to read your data into a numpy array. You can also create a synthetic one, or use some other package to read a file into python. The world is your oyster. 

In this case we will use the [`rasterio`]() package to read an [ESRI ASCII]() format file that we will download from the [OpenTopography rest server](https://opentopography.org/developers).

## Step 1: Read in a numpy array

First, we download a small patch of land near Boulder, CO. You can change the values of `north`, `south`, `east` and `west` to change the location. Its not hard to download a very large file, so increase values carefully. 

These data are provided with horizontal units of degrees. For this example we will not convert from degrees to meters, or address the issue of changing from a geographic coordinate system (WGS84) to a projected one (e.g, UTM Zone 13 N, for Colorado).

In [ ]:
west = -105.4  # longitude (degrees)
east = -105.15  # longitude (degrees)
north = 40.1  # latitude (degrees)
south = 39.9  # latitue (degrees)

URL = "http://opentopo.sdsc.edu/otr/getdem?demtype=SRTMGL3&"
url = (URL + "west=" + str(west) + "&" + "south=" + str(south) + "&"
       "east=" + str(east) + "&"
       "north=" + str(north) + "&"
       "outputFormat=AAIGrid")

f = urlopen(url)
file_like = BytesIO(f.read())
#file_like = StringIO(b.getvalue().decode("UTF-8"))

#mg, z = read_esri_ascii(file_like, name="topographic__elevation")
#imshow_grid(mg, z)

#from landlab.io import write_esri_ascii
#write_esri_ascii("example_topography.asc", mg)

We now have this thing, `file_like`...

In [ ]:
with rasterio.open(file_like) as dataset:
    #with rasterio.open('example_topography.asc') as dataset:
    nrows = dataset.height
    ncols = dataset.width
    dx, dy = dataset.res
    xy_lower_left = (dataset.bounds.left, dataset.bounds.bottom)
    elevations = dataset.read(1)



In [ ]:
plt.imshow(elevations, cmap="terrain", origin="lower")

This is upside down.... (as expected)

## Step 2: Create a ModelGrid to give to Umami

We can now create our model grid. 

In [ ]:
grid = RasterModelGrid((nrows, ncols),
                       dx=dx,
                       dy=dy,
                       xy_lower_left=xy_lower_left)

z = grid.add_field("topographic__elevation", elevations)

imshow_grid(grid, z, cmap="terrain")

Now we make our Metric.

In [ ]:
metrics = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}

#metric = Metric(grid, metrics=metrics)
#metric.calculate()

In [ ]:
#metric.names

In [ ]:
#metric.values

## Step 2: Use irregular data

We use a tenth as many points for speed here. 

In [ ]:
factor = 10

interp_obj = RegularGridInterpolator((grid.y_of_node.reshape(
    grid.shape)[:, 0], grid.x_of_node.reshape(grid.shape)[0, :]),
                                     z.reshape(grid.shape))

random_x = np.random.uniform(low=grid.x_of_node.min(),
                             high=grid.x_of_node.max(),
                             size=int(grid.x_of_node.size / factor))
random_y = np.random.uniform(low=grid.y_of_node.min(),
                             high=grid.y_of_node.max(),
                             size=int(grid.y_of_node.size / factor))
interp_z = interp_obj((random_y, random_x))

In [ ]:
vals = np.vstack((random_x, random_y, interp_z))

# sort by x then by y
sort_by_x = vals[:, vals[0, :].argsort()]
sort_by_xy = sort_by_x[:, sort_by_x[1, :].argsort()]

In [ ]:
vdg = VoronoiDelaunayGrid(sort_by_xy[0], sort_by_xy[1])
z = vdg.add_field("node", "topographic__elevation", sort_by_xy[2])

imshow_grid(vdg, z, cmap="terrain")

In [ ]:
metrics = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}

#metric = Metric(vdg, metrics=metrics)
#metric.calculate()

In [ ]:
#metric.names

In [ ]:
#metric.values

# Next steps

Now that you have a sense for how the `Metric` and `Residual` classes are used, try the next notebook: [Part 4: Example application](ExampleApplication.ipynb).